In [480]:
micro_humility_factor = 1.00   #    range from 0 (complete humility) to 1 (no humility)
macro_humility_factor = 0.96
jason_weight = .2
bruno_weight = .2
reynaldo_weight = 1 - jason_weight - bruno_weight

In [481]:
# Get ready for lots of annoying deprecation warnings
import statsmodels.api as sm

In [482]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection, preprocessing
import xgboost as xgb
import datetime
import scipy as sp
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [483]:
macro = pd.read_csv('macro.csv')
train_andy = pd.read_csv('train.csv')
test_andy = pd.read_csv('test.csv')

In [484]:
bias_list_04 = np.loadtxt("bias_list_0.4")

In [485]:
bias_list_04.astype(int)

array([ 8193, 24580, 24581, ...,  8174, 27306, 16382])

In [486]:
len(bias_list_04)

3362

In [487]:
macro["timestamp"] = pd.to_datetime(macro["timestamp"])

In [488]:
macro["year"]  = macro["timestamp"].dt.year
macro["month"] = macro["timestamp"].dt.month
macro["yearmonth"] = 100*macro.year + macro.month

In [489]:
macmeds = macro.groupby("yearmonth").median()

In [490]:
macmeds.head()

,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,eurrub,...,theaters_viewers_per_1000_cap,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,year,month
yearmonth,,,,,,,,,,,,,,,,,,,,,
201001,76.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.77500,42.5601,...,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,1
201002,72.97,NaN,NaN,315.1,337.2,NaN,18.114,NaN,30.11925,41.1596,...,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,2
201003,77.03,NaN,NaN,317.7,343.9,NaN,16.704,NaN,29.52680,40.2498,...,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,3
201004,82.87,9995.8,4.1,319.8,350.2,NaN,16.604,14.1,29.15250,39.2524,...,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,4
201005,73.05,9995.8,4.1,320.7,361.3,NaN,15.654,14.1,30.61500,38.4608,...,527.0,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,5


In [491]:
macmeds.shape 

(82, 98)

In [492]:
train_andy["timestamp"] = pd.to_datetime(train_andy["timestamp"])
train_andy["year"]  = train_andy["timestamp"].dt.year
train_andy["month"] = train_andy["timestamp"].dt.month
train_andy["yearmonth"] = 100*train_andy.year + train_andy.month
prices = train_andy[["yearmonth","price_doc"]]
p = prices.groupby("yearmonth").median()

In [493]:
p.head()

,price_doc
yearmonth,
201108,5850000
201109,5200000
201110,5500000
201111,5600000
201112,5450000


In [494]:
df = macmeds.join(p)
print df.shape

(82, 99)


In [495]:
df.head()


,oil_urals,gdp_quart,gdp_quart_growth,cpi,ppi,gdp_deflator,balance_trade,balance_trade_growth,usdrub,eurrub,...,seats_theather_rfmin_per_100000_cap,museum_visitis_per_100_cap,bandwidth_sports,population_reg_sports_share,students_reg_sports_share,apartment_build,apartment_fund_sqm,year,month,price_doc
yearmonth,,,,,,,,,,,,,,,,,,,,,
201001,76.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.77500,42.5601,...,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,1,NaN
201002,72.97,NaN,NaN,315.1,337.2,NaN,18.114,NaN,30.11925,41.1596,...,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,2,NaN
201003,77.03,NaN,NaN,317.7,343.9,NaN,16.704,NaN,29.52680,40.2498,...,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,3,NaN
201004,82.87,9995.8,4.1,319.8,350.2,NaN,16.604,14.1,29.15250,39.2524,...,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,4,NaN
201005,73.05,9995.8,4.1,320.7,361.3,NaN,15.654,14.1,30.61500,38.4608,...,0.41,993.0,NaN,NaN,63.03,22825.0,NaN,2010,5,NaN


In [496]:
import numpy.matlib as ml
 
def almonZmatrix(X, maxlag, maxdeg):
    """
    Creates the Z matrix corresponding to vector X.
    """
    n = len(X)
    Z = ml.zeros((len(X)-maxlag, maxdeg+1))
    for t in range(maxlag,  n):
       #Solve for Z[t][0].
       Z[t-maxlag,0] = sum([X[t-lag] for lag in range(maxlag+1)])
       for j in range(1, maxdeg+1):
             s = 0.0
             for i in range(1, maxlag+1):       
                s += (i)**j * X[t-i]
             Z[t-maxlag,j] = s
    return Z
# len(X)-maxlag because the first n(maxlag) value cannot be calculated for lacking of data

In [497]:
y = df.price_doc.div(df.cpi).apply(np.log).loc[201108:201506]

In [498]:
y.index

Int64Index([201108, 201109, 201110, 201111, 201112, 201201, 201202, 201203,
            201204, 201205, 201206, 201207, 201208, 201209, 201210, 201211,
            201212, 201301, 201302, 201303, 201304, 201305, 201306, 201307,
            201308, 201309, 201310, 201311, 201312, 201401, 201402, 201403,
            201404, 201405, 201406, 201407, 201408, 201409, 201410, 201411,
            201412, 201501, 201502, 201503, 201504, 201505, 201506],
           dtype='int64', name=u'yearmonth')

In [499]:
lncpi = df.cpi.apply(np.log)
tblags = 5    # Number of lags used on PDL for Trade Balance
mrlags = 5    # Number of lags used on PDL for Mortgage Rate
cplags = 5    # Number of lags used on PDL for CPI
gplags = 5
ztb = almonZmatrix(df.balance_trade.loc[201103:201506].as_matrix(), tblags, 1)
zmr = almonZmatrix(df.mortgage_rate.loc[201103:201506].as_matrix(), mrlags, 1)
zcp = almonZmatrix(lncpi.loc[201103:201506].as_matrix(), cplags, 1)
zgp = almonZmatrix(df.gdp_quart_growth.loc[201103:201506].as_matrix(), gplags, 1)
print np.shape(ztb), np.shape(zmr), np.shape(zcp), np.shape(zgp)
columns = ['tb0', 'tb1', 'mr0', 'mr1', 'cp0', 'cp1','gp0','gp1']


(47, 2) (47, 2) (47, 2) (47, 2)


In [500]:
z = pd.DataFrame( np.concatenate( (ztb, zmr, zcp,zgp), axis=1), index=y.index.values, columns=columns )

In [501]:
z.head()

,tb0,tb1,mr0,mr1,cp0,cp1,gp0,gp1
201108,104.098,268.575,73.01,184.40,35.177686,87.881303,21.6,58.5
201109,101.103,262.387,72.50,182.83,35.192806,87.947499,19.8,49.5
201110,101.265,261.268,71.70,181.11,35.201341,87.992705,21.5,49.5
201111,99.624,246.913,71.32,180.09,35.210403,88.020788,23.2,51.2
201112,101.929,245.971,70.56,178.45,35.218861,88.028984,24.9,54.6


In [502]:
X = sm.add_constant( z ) # add a column of constant

In [503]:
print X.shape, y.shape

(47, 9) (47,)


In [504]:
X.head()

,const,tb0,tb1,mr0,mr1,cp0,cp1,gp0,gp1
201108,1,104.098,268.575,73.01,184.40,35.177686,87.881303,21.6,58.5
201109,1,101.103,262.387,72.50,182.83,35.192806,87.947499,19.8,49.5
201110,1,101.265,261.268,71.70,181.11,35.201341,87.992705,21.5,49.5
201111,1,99.624,246.913,71.32,180.09,35.210403,88.020788,23.2,51.2
201112,1,101.929,245.971,70.56,178.45,35.218861,88.028984,24.9,54.6


In [505]:
# Fit macro model
eq = sm.OLS(y, X)
fit = eq.fit()

In [506]:
fit.params

const    8.202810
tb0      0.005664
tb1     -0.000099
mr0      0.003810
mr1      0.001415
cp0      0.405604
cp1     -0.159009
gp0      0.020657
gp1     -0.007527
dtype: float64

In [507]:
macmeds.shape

(82, 98)

In [508]:
df.price_doc.describe()

count    4.700000e+01
mean     6.136794e+06
std      6.065852e+05
min      4.975100e+06
25%      5.649640e+06
50%      6.100000e+06
75%      6.540318e+06
max      7.300000e+06
Name: price_doc, dtype: float64

In [509]:
df.cpi.describe()

count     81.000000
mean     408.311111
std       66.677178
min      315.100000
25%      354.000000
50%      393.000000
75%      469.400000
max      531.000000
Name: cpi, dtype: float64

In [510]:
df.cpi.loc[pd.isnull(df.cpi)]

yearmonth
201001   NaN
Name: cpi, dtype: float64

In [511]:
# Predict with macro model
test_Jason_andy_cpi = df.cpi.loc[201507:201605]
test_Jason_andy_index = test_Jason_andy_cpi.index
ztb_test_Jason_andy = almonZmatrix(df.balance_trade.loc[201502:201605].as_matrix(), tblags, 1)
zmr_test_Jason_andy = almonZmatrix(df.mortgage_rate.loc[201502:201605].as_matrix(), mrlags, 1)
zcp_test_Jason_andy = almonZmatrix(lncpi.loc[201502:201605].as_matrix(), cplags, 1)
zgp_test_Jason_andy = almonZmatrix(df.gdp_quart_growth.loc[201502:201605].as_matrix(), gplags, 1)
z_test_Jason_andy = pd.DataFrame( np.concatenate( (ztb_test_Jason_andy, zmr_test_Jason_andy, zcp_test_Jason_andy, zgp_test_Jason_andy), axis=1), 
                       test_Jason_andy_index, columns )
X_test_Jason_andy = sm.add_constant( z_test_Jason_andy )
pred_lnrp = fit.predict( X_test_Jason_andy )
pred_p = np.exp(pred_lnrp) * test_Jason_andy_cpi


In [512]:
test_andy["timestamp"] = pd.to_datetime(test_andy["timestamp"])
test_andy["year"]  = test_andy["timestamp"].dt.year
test_andy["month"] = test_andy["timestamp"].dt.month
test_andy["yearmonth"] = 100*test_andy.year + test_andy.month
test_andy_ids = test_andy[["yearmonth","id"]]

In [513]:
print test_andy.shape, test_andy_ids.shape

(7662, 294) (7662, 2)


In [514]:
monthprices = pd.DataFrame({"yearmonth":pred_p.index.values,"monthprice":pred_p.values})

In [515]:
test_andy.yearmonth.value_counts().shape

(11,)

In [516]:
pred_p.shape

(11,)

In [517]:
monthprices = pd.DataFrame({"yearmonth":pred_p.index.values,"monthprice":pred_p.values})

In [518]:
monthprices.head()

,monthprice,yearmonth
0,6.654299e+06,201507
1,6.485781e+06,201508
2,6.515985e+06,201509
3,6.377009e+06,201510
4,6.395836e+06,201511


In [519]:
test_andy_mac_price = test_andy_ids.merge(monthprices, on="yearmonth")

In [520]:
test_andy_mac_price.head()

,yearmonth,id,monthprice
0,201507,30474,6.654299e+06
1,201507,30475,6.654299e+06
2,201507,30476,6.654299e+06
3,201507,30477,6.654299e+06
4,201507,30478,6.654299e+06


In [521]:
macro_mean = np.exp(test_andy_mac_price.monthprice.apply(np.log).mean())
macro_mean
## why not get mean directly

6351983.0032084882

# Jason's Kernel


In [522]:
train_Jason = pd.read_csv('train.csv', parse_dates=['timestamp'])
test_Jason= pd.read_csv('test.csv', parse_dates=['timestamp'])
macro = pd.read_csv('macro.csv', parse_dates=['timestamp'])
id_test_Jason = test_Jason.id
train_Jason= train_Jason.drop(bias_list_04)

In [523]:
bad_index = train_Jason[train_Jason.life_sq > train_Jason.full_sq].index
train_Jason.ix[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test_Jason.ix[equal_index, "life_sq"] = test_Jason.ix[equal_index, "full_sq"]
bad_index = test_Jason[test_Jason.life_sq > test_Jason.full_sq].index
test_Jason.ix[bad_index, "life_sq"] = np.NaN
bad_index = train_Jason[train_Jason.life_sq < 5].index
train_Jason.ix[bad_index, "life_sq"] = np.NaN
bad_index = test_Jason[test_Jason.life_sq < 5].index
test_Jason.ix[bad_index, "life_sq"] = np.NaN
bad_index = train_Jason[train_Jason.full_sq < 5].index
train_Jason.ix[bad_index, "full_sq"] = np.NaN
bad_index = test_Jason[test_Jason.full_sq < 5].index
test_Jason.ix[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train_Jason.ix[kitch_is_build_year, "build_year"] = train_Jason.ix[kitch_is_build_year, "kitch_sq"]
bad_index = train_Jason[train_Jason.kitch_sq >= train_Jason.life_sq].index
train_Jason.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test_Jason[test_Jason.kitch_sq >= test_Jason.life_sq].index
test_Jason.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train_Jason[(train_Jason.kitch_sq == 0).values + (train_Jason.kitch_sq == 1).values].index
train_Jason.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = test_Jason[(test_Jason.kitch_sq == 0).values + (test_Jason.kitch_sq == 1).values].index
test_Jason.ix[bad_index, "kitch_sq"] = np.NaN
bad_index = train_Jason[(train_Jason.full_sq > 210) & (train_Jason.life_sq / train_Jason.full_sq < 0.3)].index
train_Jason.ix[bad_index, "full_sq"] = np.NaN
bad_index = test_Jason[(test_Jason.full_sq > 150) & (test_Jason.life_sq / test_Jason.full_sq < 0.3)].index
test_Jason.ix[bad_index, "full_sq"] = np.NaN

In [525]:
bad_index = train_Jason[train_Jason.life_sq > 300].index
train_Jason.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test_Jason[test_Jason.life_sq > 200].index
test_Jason.ix[bad_index, ["life_sq", "full_sq"]] = np.NaN
train_Jason.product_type.value_counts(normalize= True)
test_Jason.product_type.value_counts(normalize= True)
bad_index = train_Jason[train_Jason.build_year < 1500].index
train_Jason.ix[bad_index, "build_year"] = np.NaN
bad_index = test_Jason[test_Jason.build_year < 1500].index
test_Jason.ix[bad_index, "build_year"] = np.NaN
bad_index = train_Jason[train_Jason.num_room == 0].index 
train_Jason.ix[bad_index, "num_room"] = np.NaN
bad_index = test_Jason[test_Jason.num_room == 0].index 
test_Jason.ix[bad_index, "num_room"] = np.NaN
#bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
bad_index = [17764, 19390, 24007, 26713, 29172]
train_Jason.ix[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test_Jason.ix[bad_index, "num_room"] = np.NaN
bad_index = train_Jason[(train_Jason.floor == 0).values * (train_Jason.max_floor == 0).values].index
train_Jason.ix[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train_Jason[train_Jason.floor == 0].index
train_Jason.ix[bad_index, "floor"] = np.NaN
bad_index = train_Jason[train_Jason.max_floor == 0].index
train_Jason.ix[bad_index, "max_floor"] = np.NaN
bad_index = test_Jason[test_Jason.max_floor == 0].index
test_Jason.ix[bad_index, "max_floor"] = np.NaN
bad_index = train_Jason[train_Jason.floor > train_Jason.max_floor].index
train_Jason.ix[bad_index, "max_floor"] = np.NaN
bad_index = test_Jason[test_Jason.floor > test_Jason.max_floor].index
test_Jason.ix[bad_index, "max_floor"] = np.NaN
train_Jason.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train_Jason.ix[bad_index, "floor"] = np.NaN

In [526]:
bad_index = train_Jason[train_Jason.state == 33].index
train_Jason.ix[bad_index, "state"] = np.NaN
test_Jason.state.value_counts()

2.0    2662
1.0    2266
3.0    1913
4.0     127
Name: state, dtype: int64

In [527]:
train_Jason.loc[train_Jason.full_sq == 0, 'full_sq'] = 50
train_Jason = train_Jason[train_Jason.price_doc/train_Jason.full_sq <= 600000]
train_Jason = train_Jason[train_Jason.price_doc/train_Jason.full_sq >= 10000]

In [528]:
# Add month-year
month_year = (train_Jason.timestamp.dt.month + train_Jason.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
train_Jason['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test_Jason.timestamp.dt.month + test_Jason.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
test_Jason['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train_Jason.timestamp.dt.weekofyear + train_Jason.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
train_Jason['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test_Jason.timestamp.dt.weekofyear + test_Jason.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
test_Jason['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train_Jason['month'] = train_Jason.timestamp.dt.month
train_Jason['dow'] = train_Jason.timestamp.dt.dayofweek

test_Jason['month'] = test_Jason.timestamp.dt.month
test_Jason['dow'] = test_Jason.timestamp.dt.dayofweek

In [529]:
# Other feature engineering
train_Jason['rel_floor'] = train_Jason['floor'] / train_Jason['max_floor'].astype(float)
train_Jason['rel_kitch_sq'] = train_Jason['kitch_sq'] / train_Jason['full_sq'].astype(float)

test_Jason['rel_floor'] = test_Jason['floor'] / test_Jason['max_floor'].astype(float)
test_Jason['rel_kitch_sq'] = test_Jason['kitch_sq'] / test_Jason['full_sq'].astype(float)

train_Jason.apartment_name=train_Jason.sub_area + train_Jason['metro_km_avto'].astype(str)
test_Jason.apartment_name=test_Jason.sub_area + train_Jason['metro_km_avto'].astype(str)

train_Jason['room_size'] = train_Jason['life_sq'] / train_Jason['num_room'].astype(float)
test_Jason['room_size'] = test_Jason['life_sq'] / test_Jason['num_room'].astype(float)



In [530]:
train_Jason.shape

(27034, 299)

In [531]:
y_train = train_Jason["price_doc"]
wts = 1 - .47*(y_train == 1e6)
x_train = train_Jason.drop(["id", "timestamp", "price_doc"], axis=1)
x_test = test_Jason.drop(["id", "timestamp"], axis=1)


In [532]:
y_train.shape

(27034,)

In [533]:
wts.value_counts()

1.0    27034
Name: price_doc, dtype: int64

In [534]:
for c in x_train.columns:
    if x_train[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train[c].values)) 
        x_train[c] = lbl.transform(list(x_train[c].values))

In [535]:
for c in x_test.columns:
    if x_test[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test[c].values)) 
        x_test[c] = lbl.transform(list(x_test[c].values))

In [536]:
print x_train.shape, x_test.shape, y_train.shape

(27034, 296) (7662, 296) (27034,)


In [537]:
xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain = xgb.DMatrix(x_train, y_train, weight=wts)
dtest = xgb.DMatrix(x_test)

In [538]:
def rmsle(Truth, prediction):
    C = np.square(np.log(Truth+1)-np.log(prediction+1))
    return np.sqrt(np.sum(C)/len(Truth))

In [ ]:
#model = xgb.XGBRegressor(max_depth=5,learning_rate=0.05, subsample=0.7, colsample_bytree= 0.7, objective= 'reg:linear', silent=True )

In [ ]:
#sco_rmsle = make_scorer(rmsle, greater_is_better=True)

In [ ]:
#cv_scores = cross_val_score(model, x_train, y_train, cv=5, scoring=sco_rmsle)

In [ ]:
#cv_scores

In [ ]:
#cv_scores.mean()

In [ ]:
#cv_output = xgb.cv(xgb_params, dtrain, num_boost_round=1000, early_stopping_rounds=20,verbose_eval=50, show_stdv=False)

In [ ]:
#cv_output[['train-rmse-mean', 'test-rmse-mean']].plot()
#plt.show()

In [539]:
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=389)

In [540]:
y_predict = model.predict(dtest)

In [541]:
y_predict

array([ 6263240. ,  8377030.5,  6310842.5, ...,  4468923. ,  5846842.5,
        9196258. ], dtype=float32)

In [542]:
jason_model_output = pd.DataFrame({'id': id_test_Jason, 'price_doc': y_predict})
jason_model_output.head()

,id,price_doc
0,30474,6263240.0
1,30475,8377030.5
2,30476,6310842.5
3,30477,6305468.0
4,30478,5133986.0


In [543]:
np.exp( jason_model_output.price_doc.apply(np.log).mean() )

7330632.5399125209

# Rey's model

In [544]:

train_Rey = pd.read_csv('train.csv')
test_Rey= pd.read_csv('test.csv')
id_test = test_Rey.id

train_Rey.shape

(30471, 292)

In [545]:
y_train_Rey = train_Rey["price_doc"]
y_train_Rey = y_train_Rey.drop(bias_list_04)
train_Rey = train_Rey.drop(bias_list_04)
x_train_Rey = train_Rey.drop(["id", "timestamp", "price_doc"], axis=1)
x_test_Rey = test_Rey.drop(["id", "timestamp"], axis=1)
print y_train_Rey, x_train_Rey



0         5850000
1         6000000
2         5700000
3        13100000
4        16331452
5         9100000
6         5500000
8         5300000
10        4650000
11        4800000
12        5100000
13        5200000
14        5000000
16        6300000
17        5900000
18        7900000
19        5200000
20        8200000
21        5200000
22        6250000
23        5750000
24        6000000
26        5000000
27        4700000
28        8254400
29        5900000
30        6200000
31       18300000
32        4900000
33        4400000
           ...   
30436     6700000
30437     9037925
30438    20000000
30439     8535937
30442     4350000
30443     7567425
30444    11438948
30445    10139368
30446     6125400
30447     6373324
30448     6888144
30449     9227657
30451     2394300
30452     6800000
30453     4066740
30454     6300000
30455     9014972
30457     7800000
30458     6370777
30459     5778893
30460     9500000
30462    10544070
30463    12000000
30464    10262010
30465     

In [546]:
print y_train_Rey.shape, x_train_Rey.shape

(27109,) (27109, 289)


In [547]:
for c in x_train_Rey.columns:
    if x_train_Rey[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_train_Rey[c].values)) 
        x_train_Rey[c] = lbl.transform(list(x_train_Rey[c].values))
        
for c in x_test_Rey.columns:
    if x_test_Rey[c].dtype == 'object':
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(x_test_Rey[c].values)) 
        x_test_Rey[c] = lbl.transform(list(x_test_Rey[c].values))

In [548]:
dtrain_Rey = xgb.DMatrix(x_train_Rey, y_train_Rey)
dtest_Rey = xgb.DMatrix(x_test_Rey)

num_boost_rounds = 384 

In [549]:
model_Rey = xgb.train(dict(xgb_params, silent=0), dtrain_Rey, num_boost_round= num_boost_rounds)

In [550]:
y_predict_Rey = model_Rey.predict(dtest_Rey)

In [551]:
reynaldo_model_output = pd.DataFrame({'id': id_test, 'price_doc': y_predict_Rey})
reynaldo_model_output.head()

,id,price_doc
0,30474,6029388.0
1,30475,8668324.0
2,30476,6212551.0
3,30477,6244068.5
4,30478,5177565.0


In [552]:
np.exp( reynaldo_model_output.price_doc.apply(np.log).mean() )

7417079.083887551

# Bru's model

In [553]:
df_train = pd.read_csv("train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("macro.csv", parse_dates=['timestamp'])

In [554]:
df_train = df_train.drop(bias_list_04)

In [555]:
df_train.shape

(27109, 292)

In [556]:


df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

y_train_Bru = df_train['price_doc'].values
id_test = df_test['id']

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month + df_all.timestamp.dt.year * 100)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear + df_all.timestamp.dt.year * 100)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)


factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all_Bru = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all_Bru.shape)

X_train_Bru = X_all_Bru[:num_train]
X_test_Bru = X_all_Bru[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all_Bru = df_values.values
print(X_all_Bru.shape)

X_train_Bru = X_all_Bru[:num_train]
X_test_Bru = X_all_Bru[num_train:]

df_columns = df_values.columns


xgb_params = {
    'eta': 0.05,
    'max_depth': 5,
    'subsample': 0.7,
    'colsample_bytree': 0.7,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'silent': 1
}

dtrain_Bru = xgb.DMatrix(X_train_Bru, y_train_Bru, feature_names=df_columns)
dtest_Bru = xgb.DMatrix(X_test_Bru, feature_names=df_columns)


num_boost_round = 489  # From Bruno's original CV, I think
model_Bru = xgb.train(dict(xgb_params, silent=0), dtrain_Bru, num_boost_round=num_boost_round)

y_pred_Bru = model_Bru.predict(dtest_Bru)
bruno_model_output = pd.DataFrame({'id': id_test, 'price_doc': y_pred_Bru})
bruno_model_output.head()

(34770, 390)
(34770, 394)
(34770, 394)


,id,price_doc
0,30474,6241105.5
1,30475,8585902.0
2,30476,6391162.0
3,30477,6369486.0
4,30478,5193106.0


In [557]:
X_test_Bru.shape

(7662, 394)

# Merge and Adjust Result

In [558]:
results = reynaldo_model_output.merge( 
             jason_model_output.merge(
                 bruno_model_output, on='id', suffixes=['_jason','_bruno'] ), on='id' )
results["price_doc_reynaldo"] = results["price_doc"]

In [559]:
results.head()

,id,price_doc,price_doc_jason,price_doc_bruno,price_doc_reynaldo
0,30474,6029388.0,6263240.0,6241105.5,6029388.0
1,30475,8668324.0,8377030.5,8585902.0,8668324.0
2,30476,6212551.0,6310842.5,6391162.0,6212551.0
3,30477,6244068.5,6305468.0,6369486.0,6244068.5
4,30478,5177565.0,5133986.0,5193106.0,5177565.0


In [560]:
results["price_doc_reynaldo"] = results["price_doc"]

In [561]:
results["price_doc"] = np.exp( np.log(results.price_doc_reynaldo)*reynaldo_weight +
                               np.log(results.price_doc_jason)*jason_weight       +
                               np.log(results.price_doc_bruno)*bruno_weight          )


In [562]:
results.head()

,id,price_doc,price_doc_jason,price_doc_bruno,price_doc_reynaldo
0,30474,6117535.5,6263240.0,6241105.5,6029388.0
1,30475,8592843.0,8377030.5,8585902.0,8668324.0
2,30476,6267520.0,6310842.5,6391162.0,6212551.0
3,30477,6281234.5,6305468.0,6369486.0,6244068.5
4,30478,5171920.0,5133986.0,5193106.0,5177565.0


In [563]:
results.drop(["price_doc_reynaldo", "price_doc_bruno", "price_doc_jason"],axis=1,inplace=True)
results.head()

,id,price_doc
0,30474,6117535.5
1,30475,8592843.0
2,30476,6267520.0
3,30477,6281234.5
4,30478,5171920.0


In [564]:
def scale_miss(
        alpha,
        shifted_logs,
        oldstd,
        new_logmean
        ):
    newlogs = new_logmean + alpha*(shifted_logs - new_logmean)
    newstd = np.std(np.exp(newlogs))
    return (newstd-oldstd)**2
    

def shift_logmean_but_keep_scale(  # Or change the scale, but relative to the old scale
        data,
        new_logmean,
        rescaler
        ):
    logdata = np.log(data)
    oldstd = data.std()
    shift = new_logmean - logdata.mean()
    shifted_logs = logdata + shift
    scale = sp.optimize.leastsq( scale_miss, 1, args=(shifted_logs, oldstd, new_logmean) )
    alpha = scale[0][0]
    newlogs = new_logmean + rescaler*alpha*(shifted_logs - new_logmean)
    return np.exp(newlogs)

In [571]:
lnm = np.log(macro_mean)
y_predict = shift_logmean_but_keep_scale( results.price_doc, lnm, micro_humility_factor )
sub = pd.DataFrame({'id': id_test, 'price_doc': y_predict})
print sub.shape
print test_Jason.shape

(7662, 2)
(7662, 298)


In [572]:
sub.to_csv('test_04.csv', index=False)

# Validation for Results adjustment

In [413]:
from sklearn.model_selection import train_test_split

In [414]:
#print x_train.shape, y_train.shape

In [415]:
#x_train.head()

In [416]:
#X_train_Jv, X_test_Jv, y_train_Jv, y_test_Jv = train_test_split(x_train, y_train, test_size=0.2)

In [417]:
#print X_train_Jv.shape, X_test_Jv.shape, y_train_Jv.shape, y_test_Jv.shape

In [418]:
#df_train[df_train["life_sq"] > 7000].index   13546 must be deleted from Pick_Index
#Pick_index = list(y_test_Jv.index)
#if 13546 in Pick_index:
    #print 'Ok'
    #Pick_index.remove(13546)
#test_v_id = train.id.loc[Pick_index]


In [419]:
#test_v_id.shape

In [420]:
#y_true = y_test_Jv.ix[Pick_index]
#X_test_Jv = X_test_Jv.ix[Pick_index]

In [421]:
#y_true.shape

In [422]:
#dtest_Jv = xgb.DMatrix(X_test_Jv)

In [423]:
#y_pred_Jv = model.predict(dtest_Jv)

In [424]:
#Jv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Jv})

In [425]:
#Jv_output.shape

In [426]:
#X_test_Rv = x_train_Rey.ix[Pick_index]

In [427]:
#dtest_Rv = xgb.DMatrix(X_test_Rv)

In [428]:
#y_pred_Rv = model_Rey.predict(dtest_Rv)

In [429]:
#Rv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Rv})
#Rv_output.head()

In [430]:
#X_train_Bru

In [431]:
#x_train_Bru = pd.DataFrame(X_train_Bru)

In [432]:
#X_test_Bv = x_train_Bru.ix[Pick_index]

In [433]:
#dtest_Bv = xgb.DMatrix(X_test_Bv,feature_names=df_columns)

In [434]:
#y_pred_Bv = model_Bru.predict(dtest_Bv)

In [435]:
#Bv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Bv})
#Bv_output.head()

In [436]:
#print Rv_output.shape, Jv_output.shape, Bv_output.shape

In [437]:
#results_v = Rv_output.merge( 
             #Jv_output.merge(
                # Bv_output, on='id', suffixes=['_jason','_bruno'] ), on='id' )
#results_v["price_doc_reynaldo"] = results_v["price_doc"]

In [438]:
#results_v.head()

In [439]:
#results_v["price_doc"] = np.exp( np.log(results_v.price_doc_reynaldo)*reynaldo_weight +
                               #np.log(results_v.price_doc_jason)*jason_weight       +
                               #np.log(results_v.price_doc_bruno)*bruno_weight          )

In [440]:
#results_v.drop(["price_doc_reynaldo", "price_doc_bruno", "price_doc_jason"],axis=1,inplace=True)
#results_v.head()

In [441]:
#cv_p = p

In [442]:
#cv_p.head()

In [443]:
#cv_p['yearmonth'] = cv_p.index.values

In [444]:
#cv_p.columns= ['monthprice','yearmonth']

In [445]:
#cv_p.head()

In [446]:
#train_andy_v = train_andy[["yearmonth","id"]]

In [447]:
#train_andy_macro_= train_andy_v.merge(cv_p, on = 'yearmonth')

In [448]:
#train_andy_macro_v = train_andy_macro_.ix[Pick_index]

In [449]:
#train_andy_macro_v.shape

In [450]:
#macro_v_mean = np.exp(train_andy_macro_v.monthprice.apply(np.log).mean())

In [451]:
#macro_v_mean

In [452]:
#lnm_v = np.log(macro_v_mean)
#y_predict_v = shift_logmean_but_keep_scale( results_v.price_doc, lnm_v, micro_humility_factor )



In [453]:
#print y_predict_v.shape, y_true.shape

In [454]:
#v_score = rmsle(y_true, y_predict_v)

In [455]:
#type(y_true)

In [456]:
#type(y_predict_v)

In [457]:
#y_true.head()

In [458]:
#y_predict_v.index = y_true.index.values

In [459]:
#y_predict_v.head()

In [460]:
#y_true.shape

In [461]:
#np.shape(y_true.loc[(abs(y_predict_v-y_true)/y_true)>0.4])

In [462]:
#A = y_true.loc[abs(y_predict_v-y_true)/y_true>0.4].index.values

In [463]:
#np.shape(A)

In [464]:
jason_weight = .2
bruno_weight = .2
reynaldo_weight = 1 - jason_weight - bruno_weight 

In [465]:
import warnings
warnings.filterwarnings('ignore')
weight_val_score = []
bias_list_04=[]
cv_p = p
cv_p['yearmonth'] = cv_p.index.values
cv_p.columns= ['monthprice','yearmonth']
train_andy_v = train_andy[["yearmonth","id"]]
train_andy_macro_= train_andy_v.merge(cv_p, on = 'yearmonth')

for i in xrange(10):
    X_train_Jv, X_test_Jv, y_train_Jv, y_test_Jv = train_test_split(x_train, y_train, test_size=0.2)
    Pick_index = list(y_test_Jv.index)
    if 13546 in Pick_index:
        Pick_index.remove(13546)
    #Jason model
    test_v_id = train_Jason.id.loc[Pick_index]
    y_true = y_test_Jv.ix[Pick_index]
    X_test_Jv = X_test_Jv.ix[Pick_index]
    dtest_Jv = xgb.DMatrix(X_test_Jv)
    y_pred_Jv = model.predict(dtest_Jv)
    Jv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Jv})
    #Rey model
    X_test_Rv = x_train_Rey.ix[Pick_index]
    dtest_Rv = xgb.DMatrix(X_test_Rv)
    y_pred_Rv = model_Rey.predict(dtest_Rv)
    Rv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Rv})
    #Bru model
    x_train_Bru = pd.DataFrame(X_train_Bru)
    X_test_Bv = x_train_Bru.ix[Pick_index]
    dtest_Bv = xgb.DMatrix(X_test_Bv,feature_names=df_columns)
    y_pred_Bv = model_Bru.predict(dtest_Bv)
    Bv_output = pd.DataFrame({'id': test_v_id, 'price_doc': y_pred_Bv})
    results_v = Rv_output.merge( 
            Jv_output.merge(
                 Bv_output, on='id', suffixes=['_jason','_bruno'] ), on='id' )
    results_v["price_doc_reynaldo"] = results_v["price_doc"]
    results_v["price_doc"] = np.exp( np.log(results_v.price_doc_reynaldo)*reynaldo_weight +
                               np.log(results_v.price_doc_jason)*jason_weight       +
                               np.log(results_v.price_doc_bruno)*bruno_weight          )
    results_v.drop(["price_doc_reynaldo", "price_doc_bruno", "price_doc_jason"],axis=1,inplace=True)
    
    train_andy_macro_v = train_andy_macro_.ix[Pick_index]
    macro_v_mean = np.exp(train_andy_macro_v.monthprice.apply(np.log).mean())
    lnm_v = np.log(macro_v_mean)
    y_predict_v = shift_logmean_but_keep_scale( results_v.price_doc, lnm_v, micro_humility_factor )
    v_score = rmsle(y_true, y_predict_v)
    print v_score
    weight_val_score.append(v_score)
    y_predict_v.index = y_true.index.values
    bad_index = y_true.loc[(abs(y_predict_v-y_true)/y_true)>0.8].index.values
    bias_list_04.append(bad_index)
    


0.323767426814
0.322811359477
0.323250072372
0.32489197723
0.332298497125
0.301912695764
0.329347602453
0.34151865406
0.336478855254
0.330848769964


In [466]:
run2 = np.mean(weight_val_score)

In [467]:
print run2

0.326712591051


In [1211]:
bias_list_04[9][:5]

array([23253, 10484, 18879,  3945,  2775])

In [1212]:
y_predict_v.index

Int64Index([ 1015,  4118, 13764,  1424,  2707,  2537,  8941, 26361,  2406,
             8291,
            ...
            25485, 26069,  3140, 20773,  8319, 20629, 27445, 26798,    28,
             4002],
           dtype='int64', length=6080)

In [1213]:
sum(pd.isnull(y_predict_v))

1

In [1214]:
andy_v = train_andy.ix[bias_list_04[9]]

In [1215]:
y_predict_v = y_predict_v.loc[andy_v.index.values]

In [1216]:
y_predict_v.shape

(491,)

In [1217]:
sum(pd.isnull(y_predict_v))

0

In [1218]:
andy_v['price_doc_pred'] = y_predict_v

In [1219]:
andy_v.shape

(491, 296)

In [1224]:
A = andy_v.index.values

In [1227]:
np.savetxt(A,"bias_list_04")

ValueError: fname must be a string or file handle

In [1222]:
pd.isnull(macro.gdp_quart).value_counts()

False    2394
True       90
Name: gdp_quart, dtype: int64

In [1223]:
pd.isnull(macro.cpi).value_counts()

False    2453
True       31
Name: cpi, dtype: int64

In [ ]:
test = pd.read_csv('old_test.csv')
train = pd.read_csv('old_train.csv')
result = pd.read_csv('best.csv')
bad_index = train[train.life_sq > train.full_sq].index
train.loc[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.loc[equal_index, "life_sq"] = test.loc[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.loc[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.loc[bad_index, "life_sq"] = np.NaN

test = test[['id','sub_area','life_sq']]
test['price_doc'] = result.price_doc
test['sub_area2'] = test.sub_area
print sum(pd.isnull(train.life_sq))
print train.shape
subarea = {}
area = list(train.sub_area.value_counts()[:100].index.values)
for i in area:
    subarea[i] = int(np.median(train['price_doc'][train.sub_area == i]))

train1 = train.loc[train.life_sq.notnull()]
train1 = train1.loc[train1.life_sq!=0]
train1.sub_area.value_counts()[120]
train1['price_persq'] = train1.price_doc/train1.life_sq

subarea2 = {}
area2 = list(train1.sub_area.value_counts()[:120].index.values)
print len(area2)
for i in area2:
    subarea2[i] = int(np.median(train1['price_persq'][train1.sub_area == i]))

test['sub_area'].replace(subarea, inplace=True)  
test['sub_area2'].replace(subarea2,inplace=True)
A = list(test.sub_area.value_counts().index.values[:60])
print len(A)
for i in xrange(len(test)):
    if type(test.sub_area[i]) == int:
        if test.price_doc[i]>test.sub_area[i]*3.5:
            test.price_doc[i] = test.price_doc[i]*0.99
        if test.price_doc[i]<test.sub_area[i]*0.6:
            test.price_doc[i] = test.price_doc[i]*1.01

# 3,0.9  , 0.2,1.1   best
for i in xrange(len(test)):
    if (type(test.sub_area2[i]) == int) & pd.notnull(test.life_sq[i]):
        if test.price_doc[i]/test.life_sq[i]>test.sub_area2[i]*2.8:
            test.price_doc[i] = test.price_doc[i]*0.95
        if test.price_doc[i]/test.life_sq[i]<test.sub_area2[i]*0.3:
            test.price_doc[i] = test.price_doc[i]*1.05

test.drop(['sub_area','life_sq','sub_area2'],axis=1,inplace=True)
test.to_csv("0627_5.csv",index=False)